In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import * 
from pyspark.sql import * 

In [2]:
spark = SparkSession.builder.appName('Top100Chains').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/14 07:13:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/14 07:13:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = spark.read.csv('Top100Chains.csv',header = True)
df.show()

+----+--------------------+--------------------------+----------------+---------------+---------------+-------------+---------------+
|RANK|               CHAIN|2020 U.S. Sales ($000,000)|YOY SALES CHANGE|2020 U.S. UNITS|YOY UNIT CHANGE|      SEGMENT|  MENU CATEGORY|
+----+--------------------+--------------------------+----------------+---------------+---------------+-------------+---------------+
|   1|          McDonald's|                   $40,517|           0.30%|         13,682|         -1.20%|Quick Service|         Burger|
|   2|           Starbucks|                   $18,485|         -13.50%|         15,337|          1.90%|Quick Service|    Coffee Café|
|   3|         Chick-fil-A|                   $13,745|          13.00%|          2,659|          6.70%|Quick Service|        Chicken|
|   4|           Taco Bell|                   $11,294|           0.00%|          6,799|          0.50%|Quick Service|        Mexican|
|   5|             Wendy's|                   $10,231|        

In [4]:
df.printSchema()

root
 |-- RANK: string (nullable = true)
 |-- CHAIN: string (nullable = true)
 |-- 2020 U.S. Sales ($000,000): string (nullable = true)
 |-- YOY SALES CHANGE: string (nullable = true)
 |-- 2020 U.S. UNITS: string (nullable = true)
 |-- YOY UNIT CHANGE: string (nullable = true)
 |-- SEGMENT: string (nullable = true)
 |-- MENU CATEGORY: string (nullable = true)



In [5]:
from pyspark.sql.types import StructField,IntegerType, StructType,StringType

In [6]:
newDF=[StructField('Rank',IntegerType(),True),
       StructField('Chain',StringType(),True),
       StructField('Sales2020',StringType(),True),
       StructField('Comparison',StringType(),True),
       StructField('Unit2020',IntegerType(),True),
       StructField('UnitChange',IntegerType(),True),
       StructField('Segment',StringType(),True),
       StructField('MenuCategory',StringType(),True)
       ]
finalStruct=StructType(fields=newDF)
df=spark.read.csv('Top100Chains.csv',schema=finalStruct)

In [7]:
df.show()

+----+--------------------+--------------------+----------------+--------+----------+-------------+---------------+
|Rank|               Chain|           Sales2020|      Comparison|Unit2020|UnitChange|      Segment|   MenuCategory|
+----+--------------------+--------------------+----------------+--------+----------+-------------+---------------+
|null|               CHAIN|2020 U.S. Sales (...|YOY SALES CHANGE|    null|      null|      SEGMENT|  MENU CATEGORY|
|   1|          McDonald's|             $40,517|           0.30%|    null|      null|Quick Service|         Burger|
|   2|           Starbucks|             $18,485|         -13.50%|    null|      null|Quick Service|    Coffee Café|
|   3|         Chick-fil-A|             $13,745|          13.00%|    null|      null|Quick Service|        Chicken|
|   4|           Taco Bell|             $11,294|           0.00%|    null|      null|Quick Service|        Mexican|
|   5|             Wendy's|             $10,231|           4.80%|    nul

In [8]:
df = df.drop('Unit2020','UnitChange')

In [9]:
df.show()

+----+--------------------+--------------------+----------------+-------------+---------------+
|Rank|               Chain|           Sales2020|      Comparison|      Segment|   MenuCategory|
+----+--------------------+--------------------+----------------+-------------+---------------+
|null|               CHAIN|2020 U.S. Sales (...|YOY SALES CHANGE|      SEGMENT|  MENU CATEGORY|
|   1|          McDonald's|             $40,517|           0.30%|Quick Service|         Burger|
|   2|           Starbucks|             $18,485|         -13.50%|Quick Service|    Coffee Café|
|   3|         Chick-fil-A|             $13,745|          13.00%|Quick Service|        Chicken|
|   4|           Taco Bell|             $11,294|           0.00%|Quick Service|        Mexican|
|   5|             Wendy's|             $10,231|           4.80%|Quick Service|         Burger|
|   6|         Burger King|              $9,657|          -5.40%|Quick Service|         Burger|
|   7|             Dunkin'|             

In [10]:
import pyspark.sql.functions as f 

In [11]:
df = df.withColumn('Sales',f.translate(f.col('Sales2020'),'$,',''))

In [12]:
type(df)

pyspark.sql.dataframe.DataFrame

In [13]:
df.show()

+----+--------------------+--------------------+----------------+-------------+---------------+--------------------+
|Rank|               Chain|           Sales2020|      Comparison|      Segment|   MenuCategory|               Sales|
+----+--------------------+--------------------+----------------+-------------+---------------+--------------------+
|null|               CHAIN|2020 U.S. Sales (...|YOY SALES CHANGE|      SEGMENT|  MENU CATEGORY|2020 U.S. Sales (...|
|   1|          McDonald's|             $40,517|           0.30%|Quick Service|         Burger|               40517|
|   2|           Starbucks|             $18,485|         -13.50%|Quick Service|    Coffee Café|               18485|
|   3|         Chick-fil-A|             $13,745|          13.00%|Quick Service|        Chicken|               13745|
|   4|           Taco Bell|             $11,294|           0.00%|Quick Service|        Mexican|               11294|
|   5|             Wendy's|             $10,231|           4.80%

In [14]:
from pyspark.sql.functions import col

In [15]:
df = df.where(col("Rank").isNotNull())

In [16]:
df = df.drop('Sales2020')

In [18]:
df.show()

+----+--------------------+----------+-------------+---------------+-----+
|Rank|               Chain|Comparison|      Segment|   MenuCategory|Sales|
+----+--------------------+----------+-------------+---------------+-----+
|   1|          McDonald's|     0.30%|Quick Service|         Burger|40517|
|   2|           Starbucks|   -13.50%|Quick Service|    Coffee Café|18485|
|   3|         Chick-fil-A|    13.00%|Quick Service|        Chicken|13745|
|   4|           Taco Bell|     0.00%|Quick Service|        Mexican|11294|
|   5|             Wendy's|     4.80%|Quick Service|         Burger|10231|
|   6|         Burger King|    -5.40%|Quick Service|         Burger| 9657|
|   7|             Dunkin'|    -5.10%|Quick Service|    Coffee Cafe| 8762|
|   8|              Subway|   -18.50%|Quick Service|       Sandwich| 8318|
|   9|            Domino's|    17.60%|Quick Service|          Pizza| 8287|
|  10|Chipotle Mexican ...|     6.50%|  Fast Casual|        Mexican| 5921|
|  11|      Sonic Drive-I

In [20]:
from pyspark.sql.functions import round

In [21]:
df = df.withColumn('Comparison1',round((f.translate(f.col('Comparison'),'%','')
                                   .cast(FloatType()))/100,3)).drop('Comparison')

In [24]:
df.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Chain: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- MenuCategory: string (nullable = true)
 |-- Comparison1: double (nullable = true)
 |-- Sales1: integer (nullable = true)



In [23]:
df = df.withColumn('Sales1',df['Sales'].cast(IntegerType())).drop('Sales')

In [25]:
df.show()

+----+--------------------+-------------+---------------+-----------+------+
|Rank|               Chain|      Segment|   MenuCategory|Comparison1|Sales1|
+----+--------------------+-------------+---------------+-----------+------+
|   1|          McDonald's|Quick Service|         Burger|      0.003| 40517|
|   2|           Starbucks|Quick Service|    Coffee Café|     -0.135| 18485|
|   3|         Chick-fil-A|Quick Service|        Chicken|       0.13| 13745|
|   4|           Taco Bell|Quick Service|        Mexican|        0.0| 11294|
|   5|             Wendy's|Quick Service|         Burger|      0.048| 10231|
|   6|         Burger King|Quick Service|         Burger|     -0.054|  9657|
|   7|             Dunkin'|Quick Service|    Coffee Cafe|     -0.051|  8762|
|   8|              Subway|Quick Service|       Sandwich|     -0.185|  8318|
|   9|            Domino's|Quick Service|          Pizza|      0.176|  8287|
|  10|Chipotle Mexican ...|  Fast Casual|        Mexican|      0.065|  5921|

In [27]:
#Top 10 Sales Segment "Quick Service"
df.groupby('Segment').max('Sales1').show()

+---------------+-----------+
|        Segment|max(Sales1)|
+---------------+-----------+
|    Fine Dining|        429|
|       Midscale|       2138|
|    Fast Casual|       5921|
|  \tFast Casual|       1711|
|  Casual Dining|       3407|
|  Quick Service|      40517|
|\tQuick Service|        388|
+---------------+-----------+



In [28]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

In [30]:
windowTemp = Window.partitionBy('Segment').orderBy(col('Sales1').desc())

In [34]:
df.withColumn('row',row_number().over(windowTemp)).filter(col('row') == 1).drop('row').orderBy(col('Sales1').desc()).show()

+----+--------------------+---------------+-------------+-----------+------+
|Rank|               Chain|        Segment| MenuCategory|Comparison1|Sales1|
+----+--------------------+---------------+-------------+-----------+------+
|   1|          McDonald's|  Quick Service|       Burger|      0.003| 40517|
|  10|Chipotle Mexican ...|    Fast Casual|      Mexican|      0.065|  5921|
|  21|        Olive Garden|  Casual Dining|Italian/Pizza|     -0.205|  3407|
|  28|                IHOP|       Midscale| Family Style|     -0.346|  2138|
|  39|           Five Guys|  \tFast Casual|       Burger|       0.03|  1711|
|  85|Ruth's Chris Stea...|    Fine Dining|        Steak|      -0.39|   429|
|  93|         Taco John's|\tQuick Service|      Mexican|      0.047|   388|
+----+--------------------+---------------+-------------+-----------+------+

